# AI back test methods1

# --2 fonctions and dependancys


In [ ]:
#! python3 -m pip install --upgrade pip
#! pip install mplfinance
from xdata_config import *
from functions_module import *
import mplfinance as mpf
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.constraints import MaxNorm
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense 
from keras.models import load_model
from datetime import datetime
PDEBUG=False
def pdebug(err):
    if PDEBUG:
        print("\033[0;31m Debug msg: "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))


In [ ]:
import requests

url = 'https://api.binance.com/api/v3/ticker/price'

response = requests.get(url)
tickers = response.json()
ticker_list=[]
for ticker in tickers:
    ticker_list.append((ticker['symbol']))


# --1 Config

In [ ]:
# ai_options
Normalization_File= "False"
Model_FileName= "/UltimeTradingBot/Data/BUY_FIX/tp80_w7_max3min_Model_true_win_model_Re4.h5"
WINDOW_SIZE= 7
PRESSISION=0.0
PERIODE_START="2022-04-21 00:00:00"
PERIODE_END="2022-05-11 00:00:00"

# start_period = datetime.strptime(PERIODE_START, '%d-%m-%Y %H:%M:%S')
# end_period = datetime.strptime(PERIODE_END, '%d-%m-%Y %H:%M:%S')
start_period = pd.Timestamp(PERIODE_START)
end_period = pd.Timestamp(PERIODE_END)


#trading_options:
TAKE_PROFIT= 0.4  
STOP_LOSS=  0.8 
MAX_HOLDING_TIME= 16

USE_TRAILING_STOP_LOSS: False
TRAILING_STOP_LOSS= .002     
TRAILING_TAKE_PROFIT= .002  

PAIR_WITH= "USDT"
TRADE_TOTAL= 100 
TRADE_SLOTS= 5

TRADING_FEE= 0.1
  

In [ ]:
# Import necessary libraries
import pandas as pd
import mplfinance as mpf

# Import necessary libraries
import pandas as pd
import mplfinance as mpf

def plot_ohlcv(df,title, start_date, end_date):
    """
    Plots OHLCV data using mplfinance library.

    Args:
    df (pd.DataFrame): Dataframe containing OHLCV data with columns ['date', 'open', 'high', 'low', 'close', 'volume'].
    start_date (str): Start date of the plot in the format 'YYYY-MM-DD'.
    end_date (str): End date of the plot in the format 'YYYY-MM-DD'.
    """

    # # Ensure the index is of type datetime and sorted
    # df['date'] = pd.to_datetime(df['date'])
    # df = df.set_index('date')
    # df = df.sort_index()

    # Filter the data between start_date and end_date
    mask = (df.index >= start_date) & (df.index <= end_date)
    filtered_df = df.loc[mask]

    # Set the plot size to full width
    fig_width = 25
    fig_height = 9

    # Plot the OHLCV chart
    mpf.plot(filtered_df, type='candle', style='charles', volume=True, title=title, ylabel='Price', ylabel_lower='Volume', figratio=(fig_width, fig_height),  tight_layout=True)


# best_testing_pair="LTC/USDT"
# plot_ohlcv(df_list1h[best_testing_pair],best_testing_pair, start_date=PERIODE_START, end_date=PERIODE_END)


In [ ]:
def find_worst_period(df, window):
    """
    Find the worst period with the largest price drop in the given OHLCV DataFrame.

    Args:
    df (pd.DataFrame): Dataframe containing OHLCV data with columns ['date', 'open', 'high', 'low', 'close', 'volume'].
    window (int): Window size for calculating the largest price drop.

    Returns:
    (pd.Timestamp, pd.Timestamp): Start and end date of the worst period.
    """


    # Calculate rolling minimum close prices
    min_close = df['close'].rolling(window=window).min()

    # Find the largest price drop within the specified window
    largest_drop = (df['close'] / min_close.shift(1) - 1).idxmin()

    # Calculate start and end dates of the worst period
    start_date = largest_drop - pd.Timedelta(days=window-1)
    end_date = largest_drop

    return start_date, end_date

WORST_BTC_PERIOD=find_worst_period(df_list1h[best_testing_pair], window=10)
plot_ohlcv(df_list1h[best_testing_pair],best_testing_pair, start_date=WORST_BTC_PERIOD[0], end_date=WORST_BTC_PERIOD[1])


# --3 Importing DATA

In [ ]:
from datetime import date
#TICKERS = "../Binance-Fast-Trade-Bot/volatile_volume_" + str(date.today()) + ".txt"
TICKERS = "/UltimeTradingBot/Binance-Fast-Trade-Bot/halal_coins.txt"
VOLATILE_COINS=[line.strip() for line in open(TICKERS)]
PAIR_WITH="USDT"
VOLATILE_USDT_PAIRS=[coin+"/USDT" for coin in VOLATILE_COINS]
VOLATILE_BUSD_PAIRS=[coin+"/BUSD" for coin in VOLATILE_COINS]
for pair in VOLATILE_BUSD_PAIRS:
    if pair.replace('/' ,'') not  in ticker_list:
        print(pair)
        VOLATILE_BUSD_PAIRS.remove(pair)

for pair in VOLATILE_USDT_PAIRS:
    if pair.replace('/', '') not in ticker_list:
        print(pair)
        VOLATILE_USDT_PAIRS.remove(pair)

content = os.listdir('database/DataBackTest/1m')
for pair in VOLATILE_USDT_PAIRS:
    if pair.replace('/', '-')+'.csv' not in content:
        print(pair)
        VOLATILE_USDT_PAIRS.remove(pair)
for pair in VOLATILE_BUSD_PAIRS:
    if pair.replace('/', '-')+'.csv' not in content:
        print(pair)
        VOLATILE_BUSD_PAIRS.remove(pair)

In [ ]:
# coins_to_download=''
# for coin in LISTCOIN:
#     coins_to_download=coins_to_download+" "+coin
# os.system(f"node database/dd_for_backtest.js {coins_to_download} {PAIR_WITH}")#node database/ddargs.js ORN BUSD

In [ ]:
pair_list = VOLATILE_USDT_PAIRS
#tf = '1m'
oldest_pair = "BTC/USDT"
if oldest_pair not in pair_list: pair_list.append(oldest_pair)
df_list1m = {}
df_list1d = {}
df_list1h = {}
df_list5m = {}
df_list15m = {}


for pair in pair_list:
    df = get_backtest_historical(ccxt.binance(), pair, '1m', path="./database/")
    df_list1m[pair] = df.loc[:]

for pair in pair_list:
    df = get_backtest_historical(ccxt.binance(), pair, '1d', path="./database/")
    df_list1d[pair] = df.loc[:]

for pair in pair_list:
    df = get_backtest_historical(ccxt.binance(), pair, '1h', path="./database/")
    df_list1h[pair] = df.loc[:]

for pair in pair_list:
    df = get_backtest_historical(ccxt.binance(), pair, '5m', path="./database/")
    df_list5m[pair] = df.loc[:]

for pair in pair_list:
    df = get_backtest_historical(
        ccxt.binance(), pair, '15m', path="./database/")
    df_list15m[pair] = df.loc[:]
del(df)
df_list = df_list1m
prerr("Data load 100% use df_list1d[\"BTC/USDT\"] for exemple to access")



# --3 Decalring Tables

### Functions

In [ ]:
import pandas as pd

def mini_expand5(pair="GMT/USDT", i=0, j=10000, window=2, metadata=MetaData,
                 high_weight=1, BUY_PCT=BUY_PCT, SELL_PCT=SELL_PCT,
                 buy_function=buy_alwase):
    start_index=i
    end_index=j
    window_size=window
    buy_fn=buy_function
    """
    This function takes in several parameters to calculate technical indicators and returns a merged dataframe.
    
    :param pair: str, default "GMT/USDT"
        The trading pair to analyze.
        
    :param start_index: int, default 0
        The start index for selecting data.
        
    :param end_index: int, default 10000
        The end index for selecting data.
    
    :param window_size: int, default 2
        The window size to use for analyzing the data.
    
    :param metadata: MetaData
        The metadata to use for analyzing the data.
    
    :param high_weight: int, default 1
        The weight to use for calculating the high.
    
    :param BUY_PCT: float, default BUY_PCT
        The buy pct to use for analyzing the data.
    
    :param SELL_PCT: float, default SELL_PCT
        The sell pct to use for analyzing the data.
    
    :param buy_fn: function, default buy_min_up
        The buy function to use for analyzing the data.
    
    :return: pd.DataFrame
        A merged dataframe containing the calculated technical indicators.
    """
    pdebug(f"mini_expand : {pair}")
    # Select data
    pair_df = df_list1m[pair].iloc[start_index:end_index]
    btc_df = df_list1m["BTC/USDT"].loc[(pair_df.index[0] - pd.DateOffset(days=window_size+1)).round(freq='1 min'):pair_df.index[-1]+pd.Timedelta(f"{window_size} day")]
    # Calculate technical indicators
    pair_full = full_expand(pair_df, df_list5m[pair], df_list15m[pair], df_list1h[pair], df_list1d[pair], window_size)
    btc_full = full_expand(btc_df, df_list5m["BTC/USDT"], df_list15m["BTC/USDT"], df_list1h["BTC/USDT"], df_list1d["BTC/USDT"], window_size)   
    btc_full = btc_full.add_prefix("BTC_")
    merged = pd.merge(pair_full, btc_full, left_index=True, right_index=True)
    day_expand(merged)
    Meta_expand(merged, metadata, pair)
    merged = buy_fn(merged, BUY_PCT=BUY_PCT, SELL_PCT=SELL_PCT, window=MAX_FORCAST_SIZE)
    merged["high"] = (merged["open"] + high_weight * merged["high"] + merged["low"] + merged["close"]) / (3 + high_weight)
    merged["BTC_high"] = (merged["BTC_open"] + high_weight * merged["BTC_high"] + merged["BTC_low"] + merged["BTC_close"]) / (3 + high_weight)
    merged.rename(columns={"high":"price"},inplace=True)
    merged.rename(columns={"BTC_high":"BTC_price"},inplace=True)
    merged = merged.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])
    open_high_low_close_cols = merged.columns.str.contains("open|high|low|close")
    # merged.loc[:, open_high_low_close_cols & merged.columns.str.contains("BTC")] = (
    #     (merged["BTC_price"] - merged.loc[:, open_high_low_close_cols & merged.columns.str.contains("BTC")]) / merged["BTC_price"]
    # )
    # merged.loc[:, open_high_low_close_cols & ~merged.columns.str.contains("BTC")] = (
    #     (merged["price"] - merged.loc[:, open_high_low_close_cols & ~merged.columns.str.contains("BTC")]) / merged["price"]
    # )
    for key in merged.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
    key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            merged[key]=(merged["BTC_price"]-merged[key])/merged["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
    key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            merged[key]=(merged["price"]-merged[key])/merged["price"]

    merged=merged.dropna()
    pdebug(f'######################  mini_expand5 {pair} - shape {merged.shape}  buy mean : {hp(merged.buy.mean())} ############################')
    return merged


def create_portfolio_dataframe(num_slots, init_quantity=500, base_currency='usdt', PERIODE_START=PERIODE_START):
    # create an empty DataFrame with the columns we need
    columns = ['date', f'reserve_{base_currency}', f'total_{base_currency}']
    for i in range(num_slots):
        columns += [f'slot{i+1}_symbol', f'slot{i+1}_volume', f'slot{i+1}_original_price_{base_currency}', f'slot{i+1}_current_total_{base_currency}', f'slot{i+1}_bought_time']
    df = pd.DataFrame(columns=columns)
    # create a sample data
    for date in pd.date_range(PERIODE_START, periods=1):
        data = {'date': date, f'total_{base_currency}': init_quantity}
        for i in range(num_slots):
            data[f'slot{i+1}_bought_time'] = None
        df = df.append(data, ignore_index=True)
    # set the date column as the index of the DataFrame
    df.set_index('date', inplace=True)
    df.iloc[:, 0] = np.float64(init_quantity)
    return df.iloc[0:1]




def generate_signals(ALLTOP20VOLUMES, df_list1m, WINDOW_SIZE, MetaData, TAKE_PROFIT, STOP_LOSS, backtest_model):
    SIGNAL_DF = pd.DataFrame(columns=['coin', 'time', 'price', 'note'])
    for day, TOPLIST in ALLTOP20VOLUMES.items():
        for coin in TOPLIST:
            try:
                pdebug(f">>>>>>>>>>> working on {coin} at: {day} :")
                loc_start = df_list1m[coin].index.get_loc(day)
                loc_end = df_list1m[coin].index.get_loc(day+pd.Timedelta('1 day'))
                gc.collect()
                df = mini_expand5(pair=coin, i=loc_start, j=loc_end, window=WINDOW_SIZE, metadata=MetaData, BUY_PCT=TAKE_PROFIT, SELL_PCT=STOP_LOSS, buy_function=buy_alwase)
                dt = df.iloc[:,:-1].to_numpy(dtype=np.float32)
                predictions_note = backtest_model.predict(dt)
                predictions_round = predictions_note.round()
                dico_signal = {"coin":coin, "time":df[predictions_round==1].index.values, "price":df[predictions_round==1]["price"].values, "note":predictions_note[predictions_round==1]}
                df_signal_coin = pd.DataFrame(dico_signal)
                SIGNAL_DF = pd.concat([SIGNAL_DF, df_signal_coin])
            except:
                pdebug(f"error at {day} in {coin}")
    return SIGNAL_DF

import gc
import pandas as pd

def generate_signals_optimized(ALLTOP20VOLUMES, df_list1m, WINDOW_SIZE, MetaData, TAKE_PROFIT, STOP_LOSS, backtest_model):
    SIGNAL_DF = pd.DataFrame(columns=['coin', 'time', 'price', 'note'])
    
    # Find all unique pairs in ALLTOP20VOLUMES
    unique_pairs = set(coin for top_list in ALLTOP20VOLUMES.values() for coin in top_list)

    # Find the min and max timestamp in ALLTOP20VOLUMES
    min_timestamp = min(ALLTOP20VOLUMES)
    max_timestamp = max(ALLTOP20VOLUMES)

    # Calculate mini_expand5 and generate signals for all unique pairs in the maximum time range
    for coin in unique_pairs:
        print(f">>>>>>>>>>> working on {coin} :")
        try:
            # Compare the min and max timestamp with the index of df_list1m
            start_date = max(min_timestamp, df_list1m[coin].index[0])
            end_date = min(max_timestamp, df_list1m[coin].index[-1])

            loc_start = df_list1m[coin].index.get_loc(start_date, method='nearest')
            loc_end = df_list1m[coin].index.get_loc(end_date, method='nearest')
            
            gc.collect()
            df = mini_expand5(pair=coin, i=loc_start, j=loc_end, window=WINDOW_SIZE, metadata=MetaData, BUY_PCT=TAKE_PROFIT, SELL_PCT=STOP_LOSS, buy_function=buy_alwase)
            
            dt = df.iloc[:, :-1].to_numpy(dtype=np.float32)
            
            # Preprocessing: Handle NaN and Inf values
            dt = np.nan_to_num(dt, nan=0.0)
            dt = np.clip(dt, -1e12, 1e12)
            predictions_note = backtest_model.predict(dt)
            predictions_round = predictions_note.round()
            
            for day in ALLTOP20VOLUMES:
                        start_of_day = day.replace(hour=0, minute=0)
                        end_of_day = day.replace(hour=23, minute=59)

                        for minute in pd.date_range(start_of_day, end_of_day, freq='1T'):
                            if minute in ALLTOP20VOLUMES and coin in ALLTOP20VOLUMES[minute]:
                                signal = predictions_round[df.index.get_loc(minute)]
                                note = predictions_note[df.index.get_loc(minute)]

                                if signal == 1:
                                    SIGNAL_DF = SIGNAL_DF.append({"coin": coin, "time": minute, "price": df.loc[minute, "price"], "note": note}, ignore_index=True)


        except Exception as e:
            print(f"Error while proceding {coin} : {e}")

    return SIGNAL_DF


def get_top_volumes(start_period,end_period):
    ALLTOP20VOLUMES={}
    for day in df_list1d["BTC/USDT"].index:
        if start_period<= day <= end_period:
            Top20vol={}
            for p,df in df_list1d.items():
                if p not in ["EUR/USDT","EUR/BUSD","BTC/USDT","BTC/BUSD"]:
                    try:Top20vol.update({p:(df.loc[day].volume*df.loc[day].close)})
                    except Exception as e :pdebug(f'Time Error wile working on {p}: {e}')
                    #print (Top20vol)
            ALLTOP20VOLUMES.update({day:sorted(Top20vol, key=Top20vol.get, reverse=True)[:20]})
    return ALLTOP20VOLUMES

def is_coin_in_portfolio(symbol,instant,PORTFOLIO):
    for i in range(1,TRADE_SLOTS+1):
        try:
            if (PORTFOLIO[f"slot{i}_symbol"].loc[instant]==symbol):
                return i
        except Exception as e:
            print(f"error :{e}")
    return False

In [ ]:
portfolio_df = create_portfolio_dataframe(3)


In [ ]:
portfolio_df

In [ ]:
PORTFOLIO.columns

The code is a backtesting strategy to simulate buying and selling cryptocurrencies using a given signal DataFrame and historical prices of the cryptocurrencies.

The PORTFOLIO DataFrame is used to keep track of the state of the PORTFOLIO at each point in time during the backtesting. It contains information about the reserved and total amounts of the base currency (USDT), as well as the symbols, volumes, original prices, and current total values of up to 5 cryptocurrencies held in slots.

The fisrt_empty_slot() function checks if there is an empty slot in the PORTFOLIO where a new cryptocurrency can be added. If there is an empty slot, it returns the slot number. Otherwise, it returns False.

The buy_coin() function simulates buying a cryptocurrency at a given time and price. It checks if the cryptocurrency is not already in the PORTFOLIO and if there is enough reserved base currency to buy it. If the conditions are met, it updates the PORTFOLIO DataFrame by subtracting the base currency used for buying and adding the bought cryptocurrency to an empty slot. It also prints a message indicating the amount and symbol of the bought cryptocurrency. Otherwise, it prints a message indicating that the buying cannot be performed.

The upadate_slots() function updates the PORTFOLIO DataFrame for the current time by computing the current values of all the cryptocurrencies held in the slots using the historical prices of the cryptocurrencies. It then updates the slot information in the PORTFOLIO DataFrame.

The stop_loss_or_take_profit() function is not implemented in the provided code.

The backtest_buy_sell() function is the main function that performs the backtesting. It initializes the PORTFOLIO and ORDER_HISTORY DataFrames, defines some constants, and loops over the time range specified by the start_period and end_period parameters. At each time step, it checks the signal DataFrame for a buying or selling signal, and if there is a buying signal, it calls the buy_coin() function. If there is a selling signal, it calls the stop_loss_or_take_profit() function (not implemented) to check if the stop-loss or take-profit conditions are met, and if they are, it simulates selling the cryptocurrency by calling the sell_coin() function (not implemented). It then updates the PORTFOLIO and ORDER_HISTORY DataFrames for the current time step by calling the upadate_slots() function. Finally, it checks if the maximum holding time for each slot has been exceeded, and if it has, it simulates selling the cryptocurrency in that slot by calling the sell_coin() function (not implemented). The function returns the PORTFOLIO and ORDER_HISTORY DataFrames at the end of the backtesting.

In [63]:
ORDER_HISTORY = pd.DataFrame(columns=['Order ID', 'Pair', 'Side', 'Price', 'Quantity', 'Executed', 'Time', 'Status'])
order_id_counter = 0

def add_order_to_history(pair, side, price, quantity, executed, time, status):
    global order_id_counter
    global ORDER_HISTORY

    order_id_counter += 1
    order = {'Order ID': order_id_counter,
             'Pair': pair,
             'Side': side,
             'Price': price,
             'Quantity': quantity,
             'Executed': executed,
             'Time': time,
             'Status': status}
    ORDER_HISTORY = ORDER_HISTORY.append(order, ignore_index=True)

In [ ]:

def first_empty_slot(time, PORTFOLIO):
    for i in range(1, TRADE_SLOTS + 1):
        if pd.isna(PORTFOLIO.at[time, f'slot{i}_symbol']):
            return i
    return False



def buy_coin(time, coin, price, PORTFOLIO, pair_with_qte=TRADE_TOTAL):
    # pdebug("buy coin")
    slot_num = is_coin_in_portfolio(coin, time, PORTFOLIO)
    current_reserve = PORTFOLIO.at[time, f'reserve_{PAIR_WITH}']
    slot_ft = first_empty_slot(time, PORTFOLIO)
    
    if (not slot_num) and (current_reserve >= pair_with_qte):
        PORTFOLIO.at[time, f'reserve_{PAIR_WITH}'] = current_reserve - pair_with_qte
        PORTFOLIO.at[time, f'total_{PAIR_WITH}'] = PORTFOLIO.at[time, f'total_{PAIR_WITH}'] - TRADING_FEE * pair_with_qte / 100
        PORTFOLIO.at[time, f'slot{slot_ft}_symbol'] = coin
        PORTFOLIO.at[time, f'slot{slot_ft}_volume'] = (pair_with_qte - (TRADING_FEE * pair_with_qte / 100)) / price
        PORTFOLIO.at[time, f'slot{slot_ft}_original_price_{PAIR_WITH}'] = price
        PORTFOLIO.at[time, f'slot{slot_ft}_current_total_{PAIR_WITH}'] = price * PORTFOLIO.at[time, f'slot{slot_ft}_volume']
        PORTFOLIO.at[time, f'slot{slot_ft}_bought_time'] = time  # Add bought time for the slot
        add_order_to_history(pair=f"{coin}_{PAIR_WITH}", side='buy', price=price, quantity=pair_with_qte, executed=pair_with_qte, time=time, status='filled')
        pdebug(f"Good condition for buying {PORTFOLIO.at[time, f'slot{slot_ft}_volume']} x {coin} at {price}")
        # pdebug(PORTFOLIO.tail)
    else:
        pdebug(f"xxx We cannot buy {coin} at {price} in {time} xxx")
    return PORTFOLIO


def update_slots(time, df_list1m, PORTFOLIO):
    # pdebug("update_slots")
    if time - pd.Timedelta("1 minute") in PORTFOLIO.index:
        previous_minute = time - pd.Timedelta("1 minute")
    else:
        return PORTFOLIO  # Exit the function if previous_minute is not in the DataFrame index

    previous_reserve = PORTFOLIO.at[previous_minute, f'reserve_{PAIR_WITH}']
    previous_total = PORTFOLIO.at[previous_minute, f'total_{PAIR_WITH}']

    PORTFOLIO.at[time, f'reserve_{PAIR_WITH}'] = previous_reserve
    PORTFOLIO.at[time, f'total_{PAIR_WITH}'] = previous_total

    for slot_ft in range(1, TRADE_SLOTS + 1):
        if not pd.isna(PORTFOLIO.at[previous_minute, f'slot{slot_ft}_symbol']):
            # Get previous data
            previousslot=PORTFOLIO.at[previous_minute, f'slot{slot_ft}_symbol']
            # print(f"update_slots ->  previous slot {previousslot}")
            coin = PORTFOLIO.at[previous_minute, f'slot{slot_ft}_symbol']
            bought_at = PORTFOLIO.at[previous_minute, f'slot{slot_ft}_original_price_{PAIR_WITH}']
            previous_total_slot_value = PORTFOLIO.at[previous_minute, f'slot{slot_ft}_current_total_{PAIR_WITH}']
            slot_volume = PORTFOLIO.at[previous_minute, f'slot{slot_ft}_volume']
            # Computation 
            new_total_slot_value = df_list1m[coin].at[time, "close"] * slot_volume
            slot_value_difference = new_total_slot_value - previous_total_slot_value
            new_total_assets_value = previous_total + slot_value_difference
            pdebug(f"update_slots -> new_total_assets_value : {new_total_assets_value}")
            # Update the slot using the new price from df_list1m dataset
            PORTFOLIO.at[time, f'slot{slot_ft}_symbol'] = coin
            PORTFOLIO.at[time, f'slot{slot_ft}_original_price_{PAIR_WITH}'] = bought_at
            PORTFOLIO.at[time, f'slot{slot_ft}_volume'] = slot_volume
            PORTFOLIO.at[time, f'slot{slot_ft}_bought_time'] = PORTFOLIO.at[previous_minute, f'slot{slot_ft}_bought_time']
            PORTFOLIO.at[time, f'slot{slot_ft}_current_total_{PAIR_WITH}'] = new_total_slot_value
            PORTFOLIO.at[time, f'total_{PAIR_WITH}'] = new_total_assets_value  
            
        else:
            # print(f"no coin in slot {slot_ft}")
            pass         
    return PORTFOLIO

def stop_loss_or_take_profit(time, df_list1m, PORTFOLIO, st_pct, tp_pct, trade_fee_ptc):
    pdebug(f"stop_loss at {st_pct }% or take_profit at {tp_pct}% with trade_fee_ptc at {trade_fee_ptc}%")
    for slot_ft in range(1, TRADE_SLOTS+1):
        if pd.isna(PORTFOLIO.at[time, f"slot{slot_ft}_symbol"]):
            pdebug(f"no coin in slot {slot_ft}")
            continue
        coin = PORTFOLIO.at[time, f"slot{slot_ft}_symbol"]
        # Check if stop loss or take profit order should be triggered
        bought_price = PORTFOLIO.at[time, f"slot{slot_ft}_original_price_{PAIR_WITH}"]
        slot_volume = PORTFOLIO.at[time, f"slot{slot_ft}_volume"]
        current_price = df_list1m[coin].at[time, "close"]
        slot_total_value = PORTFOLIO.at[time, f'slot{slot_ft}_current_total_{PAIR_WITH}']  #current_price * slot_volume
        slot_fee=(trade_fee_ptc/100 * slot_total_value)
        slot_current_profit = ((current_price - bought_price) * slot_volume)# - slot_fee
        slot_current_profit_pct = (slot_current_profit / (bought_price * slot_volume)) * 100
        if slot_current_profit_pct < -st_pct:
            # Trigger stop loss
            pdebug(f"coin: {coin} in slot {slot_ft}")

            PORTFOLIO.at[time, f"slot{slot_ft}_symbol"] = np.nan
            PORTFOLIO.at[time, f"slot{slot_ft}_volume"] = np.nan
            PORTFOLIO.at[time, f"slot{slot_ft}_original_price_{PAIR_WITH}"] = np.nan
            PORTFOLIO.at[time, f"slot{slot_ft}_current_total_{PAIR_WITH}"] = np.nan
            PORTFOLIO.at[time, f'slot{slot_ft}_bought_time'] = None
            PORTFOLIO.at[time, f"total_{PAIR_WITH}"] -=   slot_fee
            PORTFOLIO.at[time, f"reserve_{PAIR_WITH}"] += (slot_total_value - slot_fee)
            add_order_to_history(pair=f"{coin}_{PAIR_WITH}", side='sell', price=current_price, quantity=slot_volume, executed=slot_volume, time=time, status='filled')
            pdebug(f"Stop loss triggered for {coin} in slot {slot_ft}")

        elif slot_current_profit_pct > tp_pct:
            # Trigger take profit
            PORTFOLIO.at[time, f"slot{slot_ft}_symbol"] = np.nan
            PORTFOLIO.at[time, f"slot{slot_ft}_volume"] = np.nan
            PORTFOLIO.at[time, f"slot{slot_ft}_original_price_{PAIR_WITH}"] = np.nan
            PORTFOLIO.at[time, f"slot{slot_ft}_current_total_{PAIR_WITH}"] = np.nan
            PORTFOLIO.at[time, f'slot{slot_ft}_bought_time'] = None
            PORTFOLIO.at[time, f"total_{PAIR_WITH}"] -=   slot_fee
            PORTFOLIO.at[time, f"reserve_{PAIR_WITH}"] += (slot_total_value - slot_fee)
            add_order_to_history(pair=f"{coin}_{PAIR_WITH}", side='sell', price=current_price, quantity=slot_volume, executed=slot_volume, time=time, status='filled')
            pdebug(f"Take profit triggered for {coin} in slot {slot_ft}")

    return PORTFOLIO


def stop_holding(time, PORTFOLIO, max_holding_duration):
    max_holding_duration_td = pd.Timedelta(minutes=max_holding_duration)

    for slot_ft in range(1, TRADE_SLOTS + 1):
        if not pd.isna(PORTFOLIO.at[time, f'slot{slot_ft}_symbol']):
            coin = PORTFOLIO.at[time, f'slot{slot_ft}_symbol']
            bought_time = PORTFOLIO.at[time, f'slot{slot_ft}_bought_time']
            holding_duration = time - bought_time

            if holding_duration > max_holding_duration_td:
                # Sell the coin
                current_price = df_list1m[coin].at[time, "close"]
                slot_volume = PORTFOLIO.at[time, f'slot{slot_ft}_volume']
                slot_total_value = PORTFOLIO.at[time, f'slot{slot_ft}_current_total_{PAIR_WITH}']
                slot_fee = (TRADING_FEE / 100) * slot_total_value
                
                PORTFOLIO.at[time, f'slot{slot_ft}_symbol'] = np.nan
                PORTFOLIO.at[time, f'slot{slot_ft}_volume'] = np.nan
                PORTFOLIO.at[time, f'slot{slot_ft}_original_price_{PAIR_WITH}'] = np.nan
                PORTFOLIO.at[time, f'slot{slot_ft}_current_total_{PAIR_WITH}'] = np.nan
                PORTFOLIO.at[time, f'slot{slot_ft}_bought_time'] = np.nan
                
                PORTFOLIO.at[time, f'total_{PAIR_WITH}'] -= slot_fee
                PORTFOLIO.at[time, f'reserve_{PAIR_WITH}'] += (slot_total_value - slot_fee)
                
                add_order_to_history(pair=f"{coin}_{PAIR_WITH}", side='sell', price=current_price, quantity=slot_volume, executed=slot_volume, time=time, status='filled')
                pdebug(f"Automatic sell triggered for {coin} in slot {slot_ft} due to exceeding holding duration")
    return PORTFOLIO




In [67]:
def trailing_stop_loss_take_profit(time, PORTFOLIO, initial_tp, initial_sl, trailing_stop_percentage, trailing_take_profit_percentage):
    global TRAILING_DATA
    for slot_ft in range(1, TRADE_SLOTS + 1):
        if not pd.isna(PORTFOLIO.at[time, f'slot{slot_ft}_symbol']):
            coin = PORTFOLIO.at[time, f'slot{slot_ft}_symbol']
            current_price = df_list1m[coin].at[time, "close"]

            if coin not in TRAILING_DATA['symbol'].values:
                TRAILING_DATA = TRAILING_DATA.append({'symbol': coin,
                                                      'trailing_stop_loss': current_price * (1 - initial_sl / 100),
                                                      'trailing_take_profit': current_price * (1 + initial_tp / 100)}, ignore_index=True)
            else:
                row_index = TRAILING_DATA[TRAILING_DATA['symbol'] == coin].index[0]
                if current_price > TRAILING_DATA.at[row_index, 'trailing_take_profit']:
                    TRAILING_DATA.at[row_index, 'trailing_take_profit'] = current_price * (1 + trailing_take_profit_percentage / 100)
                    TRAILING_DATA.at[row_index, 'trailing_stop_loss'] = current_price * (1 - trailing_stop_percentage / 100)
                elif current_price < TRAILING_DATA.at[row_index, 'trailing_stop_loss']:
                    TRAILING_DATA.at[row_index, 'trailing_stop_loss'] = current_price * (1 - trailing_stop_percentage / 100)

            trailing_stop_loss = TRAILING_DATA[TRAILING_DATA['symbol'] == coin]['trailing_stop_loss'].values[0]
            trailing_take_profit = TRAILING_DATA[TRAILING_DATA['symbol'] == coin]['trailing_take_profit'].values[0]

            if current_price <= trailing_stop_loss or current_price >= trailing_take_profit:
                # Sell the coin
                slot_volume = PORTFOLIO.at[time, f'slot{slot_ft}_volume']
                slot_total_value = PORTFOLIO.at[time, f'slot{slot_ft}_current_total_{PAIR_WITH}']
                slot_fee = (TRADING_FEE / 100) * slot_total_value

                PORTFOLIO.at[time, f'slot{slot_ft}_symbol'] = np.nan
                PORTFOLIO.at[time, f'slot{slot_ft}_volume'] = np.nan
                PORTFOLIO.at[time, f'slot{slot_ft}_original_price_{PAIR_WITH}'] = np.nan
                PORTFOLIO.at[time, f'slot{slot_ft}_current_total_{PAIR_WITH}'] = np.nan
                PORTFOLIO.at[time, f'slot{slot_ft}_bought_time'] = np.nan

                PORTFOLIO.at[time, f'total_{PAIR_WITH}'] -= slot_fee
                PORTFOLIO.at[time, f'reserve_{PAIR_WITH}'] += (slot_total_value - slot_fee)

                add_order_to_history(pair=f"{coin}_{PAIR_WITH}", side='sell', price=current_price, quantity=slot_volume, executed=slot_volume, time=time, status='filled')
                pdebug(f"Trailing stop loss or take profit triggered for {coin} in slot {slot_ft}")
    return PORTFOLIO


In [69]:
SORT_SIGNAL=True
def backtest_buy_sell(SIGNAL_DF, df_list1m, start_period, end_period):
    pdebug(f"---   Backtesting form {start_period} to {end_period} while tp:{TAKE_PROFIT}% and sl:{STOP_LOSS}% and trading fee: {TRADING_FEE}% --- ")
    # Initialize variables and counters
    global ORDER_HISTORY 
    ORDER_HISTORY = pd.DataFrame(columns=['Order ID', 'Pair', 'Side', 'Price', 'Quantity', 'Executed', 'Time', 'Status'])
    current_time = pd.Timestamp(start_period)
    end_period_margin = end_period + pd.Timedelta(f"{2 * MAX_HOLDING_TIME} minute")
    PORTFOLIO = create_portfolio_dataframe(TRADE_SLOTS, init_quantity=550, base_currency=PAIR_WITH, PERIODE_START=start_period)
    PeriodRange = pd.date_range(start=pd.Timestamp(start_period), end=pd.Timestamp(end_period), freq='1min')

    # Verify buying opportunities
    for current_time in PeriodRange:
        if current_time == pd.Timestamp(start_period):
            pdebug(f"update_slots -> The first case : {current_time}")
            PORTFOLIO.loc[current_time + pd.Timedelta("1 minute")] =PORTFOLIO.loc[current_time]
            continue
        else:
            previous_minute = current_time - pd.Timedelta("1 minute")
        # print(PORTFOLIO.tail)
        PORTFOLIO=update_slots(current_time, df_list1m, PORTFOLIO=PORTFOLIO)
        pdebug(current_time)
        if SORT_SIGNAL:
            minute_signal_list=SIGNAL_DF[SIGNAL_DF.time == pd.Timestamp(current_time)].sort_values('note',ascending=False)
        else:
            minute_signal_list=SIGNAL_DF[SIGNAL_DF.time == pd.Timestamp(current_time)]
        for i, sig_row in minute_signal_list.iterrows():
            # Buy the coin if slot and cash is available:
            coin = sig_row['coin']
            time = sig_row['time']
            price = sig_row['price']  # We assume there's only one price per signal
            note = sig_row['note']
            slot_num = is_coin_in_portfolio(coin, previous_minute, PORTFOLIO)

            # Buy first:
            if (note + PRECISION) >= 0.5:
                PORTFOLIO = buy_coin(time, coin, price, PORTFOLIO=PORTFOLIO, pair_with_qte=TRADE_TOTAL)

        if USE_TRAILING_STOP_LOSS:
            PORTFOLIO = trailing_stop_loss_take_profit(current_time, PORTFOLIO, TAKE_PROFIT, STOP_LOSS, TRAILING_STOP_LOSS, TRAILING_TAKE_PROFIT)
        else:
            PORTFOLIO = stop_loss_or_take_profit(current_time, df_list1m, PORTFOLIO, STOP_LOSS, TAKE_PROFIT, TRADING_FEE)
        PORTFOLIO = stop_holding(current_time, PORTFOLIO, MAX_HOLDING_TIME)
        pct_benefit = 100 * PORTFOLIO.at[current_time, f'total_{PAIR_WITH}'] / PORTFOLIO[f'total_{PAIR_WITH}'].iloc[0]
        pdebug(f"Total benefit pct {pct_benefit}")
        
    return PORTFOLIO




In [ ]:


import matplotlib.pyplot as plt

def back_test_metrics_v1(PORTFOLIO):
    initial_investment = PORTFOLIO.at[PORTFOLIO.index[0], f"reserve_{PAIR_WITH}"]
    final_value = PORTFOLIO.at[PORTFOLIO.index[-1], f"total_{PAIR_WITH}"]
    profit = final_value - initial_investment
    roi = (profit / initial_investment) * 100

    print(f"Initial Investment: {initial_investment:.2f} {PAIR_WITH}")
    print(f"Final Value: {final_value:.2f} {PAIR_WITH}")
    print(f"Profit: {profit:.2f} {PAIR_WITH}")
    print(f"Return on Investment (ROI): {roi:.2f}%")

    # Draw plot of total assets
    plt.plot(PORTFOLIO.index, PORTFOLIO[f"total_{PAIR_WITH}"])
    plt.xlabel("Time")
    plt.ylabel(f"Total {PAIR_WITH}")
    plt.title("Total Assets Value Over Time")
    plt.grid()
    plt.show()

# Example usage:
# back_test_metrics(PORTFOLIO)

import matplotlib.pyplot as plt

import matplotlib.pyplot as plt

def back_test_metrics(PORTFOLIO):
    initial_investment = PORTFOLIO.at[PORTFOLIO.index[0], f"reserve_{PAIR_WITH}"]
    final_value = PORTFOLIO.at[PORTFOLIO.index[-1], f"total_{PAIR_WITH}"]
    profit = final_value - initial_investment
    roi = (profit / initial_investment) * 100

    print(f"Initial Investment: {initial_investment:.2f} {PAIR_WITH}")
    print(f"Final Value: {final_value:.2f} {PAIR_WITH}")
    print(f"Profit: {profit:.2f} {PAIR_WITH}")
    print(f"Return on Investment (ROI): {roi:.2f}%")

    # Calculate the 10 biggest wins and losses
    PORTFOLIO['change'] = PORTFOLIO[f"total_{PAIR_WITH}"].diff()
    PORTFOLIO['change'] = pd.to_numeric(PORTFOLIO['change'], errors='coerce')
    top_10_wins = PORTFOLIO.nlargest(10, 'change')
    top_10_losses = PORTFOLIO.nsmallest(10, 'change')

    # Draw plot of total assets
    plt.plot(PORTFOLIO.index, PORTFOLIO[f"total_{PAIR_WITH}"], label="Total Assets")
    plt.scatter(top_10_wins.index, top_10_wins[f"total_{PAIR_WITH}"], color='g', label='Top 10 Wins')
    plt.scatter(top_10_losses.index, top_10_losses[f"total_{PAIR_WITH}"], color='r', label='Top 10 Losses')
    plt.xlabel("Time")
    plt.ylabel(f"Total {PAIR_WITH}")
    plt.title("Total Assets Value Over Time")
    plt.legend()
    plt.grid()
    plt.show()

# Example usage:
# back_test_metrics(PORTFOLIO)
def calculate_wins_losses(trade_history):
    wins = 0
    losses = 0

    open_positions = {}

    for index, trade in trade_history.iterrows():
        symbol = trade['Pair'].split('_')[0]
        side = trade['Side']
        price = trade['Price']

        if side == 'buy':
            open_positions[symbol] = price
        elif side == 'sell' and symbol in open_positions:
            buy_price = open_positions[symbol]

            if price > buy_price:
                wins += 1
            else:
                losses += 1

            del open_positions[symbol]
    print(f"total trades {wins+losses} whith {wins} win ({(100*wins/(wins+losses)):.2f}%) and {losses} losses ({(100*losses/(wins+losses)):.2f}%) ")
    return wins, losses


In [ ]:
def get_back_test_metrics(PORTFOLIO):
    initial_investment = PORTFOLIO.at[PORTFOLIO.index[0], f"reserve_{PAIR_WITH}"]
    final_value = PORTFOLIO.at[PORTFOLIO.index[-1], f"total_{PAIR_WITH}"]
    profit = final_value - initial_investment
    roi = (profit / initial_investment) * 100

    # Calculate the 10 biggest wins and losses
    PORTFOLIO['change'] = PORTFOLIO[f"total_{PAIR_WITH}"].diff()
    PORTFOLIO['change'] = pd.to_numeric(PORTFOLIO['change'], errors='coerce')
    top_10_wins = PORTFOLIO.nlargest(10, 'change')
    top_10_losses = PORTFOLIO.nsmallest(10, 'change')

    metrics = {
        'initial_investment': initial_investment,
        'final_value': final_value,
        'profit': profit,
        'roi': roi,
        'top_10_wins': top_10_wins,
        'top_10_losses': top_10_losses
    }

    return metrics

# # Call the function and get the metrics dictionary
# metrics = get_back_test_metrics(PORTFOLIO)

# # Print the metrics
# print(f"Initial Investment: {metrics['initial_investment']:.2f} {PAIR_WITH}")
# print(f"Final Value: {metrics['final_value']:.2f} {PAIR_WITH}")
# print(f"Profit: {metrics['profit']:.2f} {PAIR_WITH}")
# print(f"Return on Investment (ROI): {metrics['roi']:.2f}%")

# # Use the metrics dictionary in other parts of your code
# top_10_wins = metrics['top_10_wins']
# top_10_losses = metrics['top_10_losses']


In [ ]:
# ai_options
Normalization_File= "False"
Model_FileName= "/UltimeTradingBot/Data/BUY_OPTIMAL/tp150_w15_max7min_Model_VeryDeep.h5"
WINDOW_SIZE= 15
PRESSISION=0.0
PERIODE_START="2022-04-21 00:00:00"
PERIODE_END="2022-04-22 00:00:00"

# start_period = datetime.strptime(PERIODE_START, '%d-%m-%Y %H:%M:%S')
# end_period = datetime.strptime(PERIODE_END, '%d-%m-%Y %H:%M:%S')
start_period = pd.Timestamp(PERIODE_START)
end_period = pd.Timestamp(PERIODE_END)


#trading_options:
TAKE_PROFIT= 1.5  
STOP_LOSS=  5
MAX_HOLDING_TIME= 7#60*24

USE_TRAILING_STOP_LOSS= False
TRAILING_STOP_LOSS= .002     
TRAILING_TAKE_PROFIT= .002  

PAIR_WITH= "USDT"
TRADE_TOTAL= 100 
TRADE_SLOTS= 5

TRADING_FEE= 0.1
PRECISION=0.0




  
  
plot_ohlcv(df_list1h["BTC/USDT"],"BTC/USDT", start_date=start_period, end_date=end_period)

ORDER_HISTORY= pd.DataFrame(columns= ['Order ID', 'Pair', 'Side', 'Price', 'Quantity', 'Executed', 'Time', 'Status'])
ALLTOP20VOLUMES=get_top_volumes(start_period-pd.Timedelta("10 minute") ,end_period+pd.Timedelta("10 minute"))
backtest_model=load_model(Model_FileName)
SIGNAL_DF = generate_signals(ALLTOP20VOLUMES, df_list1m, WINDOW_SIZE, MetaData, TAKE_PROFIT, STOP_LOSS, backtest_model)



In [71]:
TAKE_PROFIT= 0.5  
STOP_LOSS=  10.5 
MAX_HOLDING_TIME= 60*24

USE_TRAILING_STOP_LOSS= False

TRAILING_DATA=pd.DataFrame()
PORTFOLIO=backtest_buy_sell(SIGNAL_DF, df_list1m, start_period=start_period, end_period=end_period)
back_test_metrics(PORTFOLIO)
metrics=get_back_test_metrics(PORTFOLIO)
win,loss=calculate_wins_losses(ORDER_HISTORY)


In [70]:
ORDER_HISTORY=None

### Comparing results
for tp1.5 sl10.5 best results of MAX_HOLDING_TIME is 24 hour with 4 hours can be concidired a decent time  with roi of -3.7 vs -0.25
 
#### comparing medels



TAKE_PROFIT= 2.9  
STOP_LOSS=  8.8 

Initial Investment: 550.00 USDT
Final Value: 546.41 USDT
Profit: -3.59 USDT
Return on Investment (ROI): -0.65%


vs

Final Value: 550.87 USDT
Profit: 0.87 USDT
Return on Investment (ROI): 0.16%

In [ ]:
SIGNAL_DF[SIGNAL_DF.time=='2022-04-23 00:07:00	'].sort_values('note',ascending=False)

In [ ]:
calculate_wins_losses(ORDER_HISTORY)

In [ ]:
import numpy as np

def tuner(SIGNAL_DF, df_list1m, start_period, end_period, tp_range, sl_range, mht_range):
    best_roi = -np.inf
    best_params = None
    best_metrics = None

    for tp in np.arange(*tp_range):
        for sl in np.arange(*sl_range):
            for mht in np.arange(*mht_range):
                global TAKE_PROFIT, STOP_LOSS, MAX_HOLDING_TIME
                TAKE_PROFIT = tp
                STOP_LOSS = sl
                MAX_HOLDING_TIME = mht

                PORTFOLIO = backtest_buy_sell(SIGNAL_DF, df_list1m, start_period=start_period, end_period=end_period)
                metrics = get_back_test_metrics(PORTFOLIO)

                if metrics['roi'] > best_roi:
                    best_roi = metrics['roi']
                    best_params = (tp, sl, mht)
                    best_metrics = metrics
                print('----------------------------------')
                print(f'for parameters TAKE_PROFIT:{TAKE_PROFIT} STOP_LOSS:{STOP_LOSS} MAX_HOLDING_TIME:{MAX_HOLDING_TIME} -> the ROI: {metrics["roi"]:.2f} , the profit: {metrics["profit"]:.2f} ,the final {metrics["final_value"]:.2f}  ')
                print('----------------------------------')
    return best_params, best_metrics

# Set the ranges for the tuner function
tp_range = (0.4, 2.5, 0.3)  # start, end, step
sl_range = (0.4, 5.0, 0.6)
mht_range = (7, 60 * 4, 5)  # 1 hour to 2 days, step by 1 hour

best_params, best_metrics = tuner(SIGNAL_DF, df_list1m, start_period, end_period, tp_range, sl_range, mht_range)

print("Best Parameters:")
print(f"Take Profit: {best_params[0]}")
print(f"Stop Loss: {best_params[1]}")
print(f"Max Holding Time: {best_params[2]}")

print("Best Metrics:")
print(f"Initial Investment: {best_metrics['initial_investment']:.2f} {PAIR_WITH}")
print(f"Final Value: {best_metrics['final_value']:.2f} {PAIR_WITH}")
print(f"Profit: {best_metrics['profit']:.2f} {PAIR_WITH}")
print(f"Return on Investment (ROI): {best_metrics['roi']:.2f}%")
